In [19]:
import numpy as np
import pandas as pd
from econml.grf import CausalForest, CausalIVForest, RegressionForest
from econml.dml import CausalForestDML
import scipy.special
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [20]:
#url_1 = r"https://www.dropbox.com/s/up0k1i6btb8qmb7/usa_00004.csv?dl=1"
url_1 = r"C:\Users\Krishna\Dropbox\usa_00004.csv"
data_2019 = pd.read_csv(url_1)
data_2019.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,COUNTYFIP,METRO,STRATA,GQ,...,MARST,RACE,RACED,EDUC,EDUCD,EMPSTAT,EMPSTATD,INCTOT,SEX_SP,AGE_SP
0,2019,201901,1,2019010000088,11.0,2019000000011,0,1,220001,4,...,6,2,200,4,40,3,30,9000,NaN,NaN
1,2019,201901,2,2019010000096,70.0,2019000000021,0,1,100001,3,...,6,1,100,4,40,3,30,150,NaN,NaN
2,2019,201901,3,2019010000153,20.0,2019000000031,15,4,110001,4,...,6,2,200,7,71,1,10,1400,NaN,NaN
3,2019,201901,4,2019010000198,79.0,2019000000041,15,4,110001,3,...,5,1,100,3,30,3,30,22700,NaN,NaN
4,2019,201901,5,2019010000205,53.0,2019000000051,97,4,270101,3,...,3,2,200,3,30,3,30,0,NaN,NaN


In [21]:
#url_2 = r"https://www.dropbox.com/s/yep1ue5e4u2qhc3/usa_00005.csv?dl=1"
url_2 = r"C:\Users\Krishna\Dropbox\usa_00005.csv"
data_2015 = pd.read_csv(url_2)
data_2015.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,COUNTYFIP,METRO,STRATA,GQ,...,MARST,RACE,RACED,EDUC,EDUCD,EMPSTAT,EMPSTATD,INCTOT,SEX_SP,AGE_SP
0,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,1,2,200,5,50,1,10,3450,2.0,61.0
1,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,1,2,200,7,71,3,30,14000,1.0,56.0
2,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,6,2,200,1,15,0,0,9999999,NaN,NaN
3,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,4,2,200,7,71,1,10,35000,NaN,NaN
4,2015,201501,2,160,65.0,2015000000021,0,1,100001,1,...,4,8,802,2,26,3,30,8800,NaN,NaN


In [22]:
data_2015.columns


Index(['YEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'COUNTYFIP',
       'METRO', 'STRATA', 'GQ', 'SSMC', 'PERNUM', 'PERWT', 'SEX', 'AGE',
       'MARST', 'RACE', 'RACED', 'EDUC', 'EDUCD', 'EMPSTAT', 'EMPSTATD',
       'INCTOT', 'SEX_SP', 'AGE_SP'],
      dtype='object')

In [23]:
#concat dataset
frames = [data_2015, data_2019]
final_df = pd.concat(frames)
final_df.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,COUNTYFIP,METRO,STRATA,GQ,...,MARST,RACE,RACED,EDUC,EDUCD,EMPSTAT,EMPSTATD,INCTOT,SEX_SP,AGE_SP
0,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,1,2,200,5,50,1,10,3450,2.0,61.0
1,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,1,2,200,7,71,3,30,14000,1.0,56.0
2,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,6,2,200,1,15,0,0,9999999,NaN,NaN
3,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,4,2,200,7,71,1,10,35000,NaN,NaN
4,2015,201501,2,160,65.0,2015000000021,0,1,100001,1,...,4,8,802,2,26,3,30,8800,NaN,NaN


In [24]:
final_df = final_df.dropna()
final_df.head()
final_df.shape

(2910350, 25)

In [25]:
final_df['SEX_SP'] = final_df['SEX_SP'].astype(int)

final_df.loc[:, 'same_sex_couple'] = 0
final_df.loc[final_df['SEX'] == final_df['SEX_SP'], 'same_sex_couple'] = 1

In [26]:
check_data = final_df[['SEX',
                        'SEX_SP',
                        'same_sex_couple']].copy()
check_data

,SEX,SEX_SP,same_sex_couple
0,1,2,0
1,2,1,0
6,1,2,0
7,2,1,0
11,2,1,0
...,...,...,...
3239538,2,1,0
3239546,1,2,0
3239547,2,1,0
3239551,2,1,0


In [27]:
final_df["MARST"].value_counts()

1    2621434
6     173648
4      94959
5      11868
3       8441
Name: MARST, dtype: int64

In [28]:
final_df.loc[:, 'married_couple'] = 0
final_df.loc[final_df['MARST'] <= 1, 'married_couple'] = 1
final_df["married_couple"].value_counts()

1    2621434
0     288916
Name: married_couple, dtype: int64

In [29]:
final_df["YEAR"].unique()
final_df = final_df[final_df["AGE"] > 18] 

In [31]:
final_df.shape

(2908911, 27)

In [32]:
features = final_df[['SEX', 'AGE', 'RACE', 'EDUC', 'EMPSTAT']]
treatment = final_df['married_couple']
instrument = final_df['INCTOT']

In [33]:
final_df['treat'] = final_df.apply(lambda _:0, axis = 1)